In [1]:
import pandas as pd
import numpy as np
import os
import Levenshtein
from pprint import pprint
import matplotlib as plt

### Obtendo lista de municípios via dados do IBGE (Censo de 2010)  

In [2]:
LISTA_ESTADOS=(
    ('AC', 'Acre'),
    ('AL', 'Alagoas'),
    ('AP', 'Amapá'),
    ('AM', 'Amazonas'),
    ('BA', 'Bahia'),
    ('CE', 'Ceará'),
    ('DF', 'Distrito Federal'),
    ('ES', 'Espírito Santo'),
    ('GO', 'Goiás'),
    ('MA', 'Maranhão'),
    ('MT', 'Mato Grosso'),
    ('MS', 'Mato Grosso do Sul'),
    ('MG', 'Minas Gerais'),
    ('PA', 'Pará'),
    ('PB', 'Paraíba'),
    ('PR', 'Paraná'),
    ('PE', 'Pernambuco'),
    ('PI', 'Piauí'),
    ('RJ', 'Rio de Janeiro'),
    ('RN', 'Rio Grande do Norte'),
    ('RS', 'Rio Grande do Sul'),
    ('RO', 'Rondônia'),
    ('RR', 'Roraima'),
    ('SC', 'Santa Catarina'),
    ('SP', 'São Paulo'),
    ('SE', 'Sergipe'),
    ('TO', 'Tocantins'),
    )


Lendo os 27 arquivos do Censo (um por UF) para DataFrames, e os concatenando.

In [3]:
lista_DFs=[]
for i, file in enumerate(os.listdir('./ibge/')):
    tmp = pd.read_excel(os.path.join('ibge',file), convert_float=True)
    tmp['UF'] = LISTA_ESTADOS[i][0]
    lista_DFs.append(tmp)

In [4]:
df_municipios = pd.concat(lista_DFs)

In [5]:
len(df_municipios)

5619

In [6]:
len(df_municipios[df_municipios['Nome do município'].isnull()])

27

In [7]:
df_municipios.dropna(inplace=True)

In [8]:
df_municipios.rename(columns={"Código do município": "CodMunicipio",
                              "Nome do município": "NomeMunicipio",
                              "Total da população 2000": "TotalPopulacao2000",
                              "Total de homens": "TotalHomens",
                              "Total de mulheres": "TotalMulheres",
                              "Total da população urbana": "TotalPopulacaoUrbana",
                              "Total da população rural": "TotalPopulacaoRural",
                              "Total da população 2010": "TotalPopulacao2010"}, inplace=True)

Convertendo colunas de float para int:

In [9]:
for i in df_municipios.columns:
    if df_municipios[i].dtypes == 'float64':
        df_municipios[i] = df_municipios[i].astype(np.int64)

In [10]:
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5592 entries, 0 to 139
Data columns (total 9 columns):
CodMunicipio            5592 non-null object
NomeMunicipio           5592 non-null object
TotalPopulacao2000      5592 non-null int64
TotalHomens             5592 non-null int64
TotalMulheres           5592 non-null int64
TotalPopulacaoUrbana    5592 non-null int64
TotalPopulacaoRural     5592 non-null int64
TotalPopulacao2010      5592 non-null int64
UF                      5592 non-null object
dtypes: int64(6), object(3)
memory usage: 436.9+ KB


In [11]:
df_estados = df_municipios[df_municipios.CodMunicipio=="Total"].copy()

In [12]:
df_municipios = df_municipios[df_municipios.CodMunicipio!="Total"].copy()

In [13]:
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 138
Data columns (total 9 columns):
CodMunicipio            5565 non-null object
NomeMunicipio           5565 non-null object
TotalPopulacao2000      5565 non-null int64
TotalHomens             5565 non-null int64
TotalMulheres           5565 non-null int64
TotalPopulacaoUrbana    5565 non-null int64
TotalPopulacaoRural     5565 non-null int64
TotalPopulacao2010      5565 non-null int64
UF                      5565 non-null object
dtypes: int64(6), object(3)
memory usage: 434.8+ KB


In [14]:
df_municipios.TotalPopulacao2010.sum()

190732694

Os dados do Censo são de 2010:
* Total de 5565 municípios;
* População total do país: 190.732.694;


In [15]:
df_municipios.sort_values('TotalPopulacao2010', ascending=False, inplace=True)

In [16]:
df_municipios = df_municipios[['CodMunicipio', 'NomeMunicipio', 'UF', 'TotalPopulacao2010', 'TotalPopulacao2000',
                               'TotalHomens', 'TotalMulheres', 'TotalPopulacaoUrbana', 'TotalPopulacaoRural']]

In [17]:
df_municipios.head()

,CodMunicipio,NomeMunicipio,UF,TotalPopulacao2010,TotalPopulacao2000,TotalHomens,TotalMulheres,TotalPopulacaoUrbana,TotalPopulacaoRural
564,3550308,São Paulo,SP,11244369,10434252,5323385,5920984,11125243,119126
67,3304557,Rio de Janeiro,RJ,6323037,5857904,2960954,3362083,6323037,0
335,2927408,Salvador,BA,2676606,2443107,1249301,1427305,2675875,731
0,5300108,Brasília,DF,2562963,2051146,1225237,1337726,2476249,86714
58,2304400,Fortaleza,CE,2447409,2141402,1145799,1301610,2447409,0


---

### Lendo tabela do Bolsa Família (06/2016) 

In [406]:
dfBolsa = pd.read_csv('201606_BolsaFamiliaFolhaPagamento.csv',
                      sep='\t',
                      header=(0),
                      encoding='cp1252',
                      dtype={"Valor Parcela": np.str,
                             "Código SIAFI Município": np.str})

In [407]:
dfBolsa.rename(columns={"Código SIAFI Município": "CodigoSIAFImunicipio",
                        "Nome Município": "NomeMunicipio",
                        "Código Função": "CodigoFuncao",
                        "Código Subfunção": "CodigoSubfuncao",
                        "Código Programa": "CodigoPrograma",
                        "Código Ação": "CodigoAcao",
                        "NIS Favorecido": "NISFavorecido",
                        "Nome Favorecido": "NomeFavorecido",
                        "Fonte-Finalidade": "FonteFinalidade",
                        "Valor Parcela": "ValorParcela",
                        "Mês Competência": "MesCompetencia"}, inplace=True)

In [408]:
dfBolsa.ValorParcela = dfBolsa.ValorParcela.str.replace(",","")

In [409]:
dfBolsa.ValorParcela = dfBolsa.ValorParcela.apply(pd.to_numeric)

In [410]:
dfBolsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13849866 entries, 0 to 13849865
Data columns (total 12 columns):
UF                      object
CodigoSIAFImunicipio    object
NomeMunicipio           object
CodigoFuncao            int64
CodigoSubfuncao         int64
CodigoPrograma          int64
CodigoAcao              int64
NISFavorecido           int64
NomeFavorecido          object
FonteFinalidade         object
ValorParcela            float64
MesCompetencia          object
dtypes: float64(1), int64(5), object(6)
memory usage: 1.2+ GB


In [411]:
len(dfBolsa[dfBolsa.ValorParcela>1000])

304

In [412]:
dfBolsa.ValorParcela.sum()

2237424572.0

In [413]:
dfBolsa.ValorParcela.mean()

161.54846350137973

In [414]:
dfBolsa.ValorParcela.count()

13849866

* 304 pessoas receberam mais de R\$ 1000 no mês de junho do Programa Bolsa-Família;
* O total de repasse no mês foi de mais de 2 bilhões de reais (R\$ 2.237.424.572,00);
* A média dos pagamentos é de R\$ 161,55, e a quantidade total de pagamentos 13.849.866;


In [415]:
del dfBolsa['FonteFinalidade']

In [416]:
dfBolsa.head()

,UF,CodigoSIAFImunicipio,NomeMunicipio,CodigoFuncao,CodigoSubfuncao,CodigoPrograma,CodigoAcao,NISFavorecido,NomeFavorecido,ValorParcela,MesCompetencia
0,SP,6291,CAMPINAS,8,244,1335,8442,13287063897,ANGELA APARECIDA SERAFIM,182.0,06/2016
1,BA,3389,BROTAS DE MACAUBAS,8,244,1335,8442,16074176737,FLAVIANO SEBASTIAO DOS SANTOS,105.0,06/2016
2,AM,0255,MANAUS,8,244,1335,8442,12953494024,GLEICIANE SOUZA OLIVEIRA,217.0,06/2016
3,PE,2531,RECIFE,8,244,1335,8442,20033347012,ELANE PATRICIA DA SILVA DAMASIO NUNES,154.0,06/2016
4,MA,0755,CARUTAPERA,8,244,1335,8442,16099396730,EDNA MARIA FERREIRA PINHEIRO,77.0,06/2016


In [417]:
dfBolsa.shape

(13849866, 11)

#### Realizando o groupby por município

In [295]:
dfBolsaTmp = dfBolsa.groupby('CodigoSIAFImunicipio')

In [296]:
dfBolsaGrouped = dfBolsaTmp['ValorParcela'].agg({'ValorTotal':np.sum, 'ValorMedio':np.mean})

In [297]:
dfBolsaGrouped['Count'] = dfBolsaTmp.size()

In [305]:
dfBolsaGrouped.reset_index(inplace=True)

In [306]:
dfBolsaGrouped.head()

,CodigoSIAFImunicipio,ValorMedio,ValorTotal,Count
0,0001,156.740193,583387.0,3722
1,0002,144.883144,140102.0,967
2,0003,145.421862,3517464.0,24188
3,0004,139.553310,400518.0,2870
4,0005,137.725603,644969.0,4683


In [426]:
len(dfBolsaGroupedroupedrouped)

5570

### Gerando tabela auxiliar para relacionar os municípios do Censo do IBGE com os da lista do Bolsa-Família

Nesse trecho, criei um DataFrame para relacionar os municípios do Censo com os municípios que aparecem na relação do Bolsa-Família.

Como os códigos que os identificam não são os mesmos entre os 2 datasets, foi necessário realizar o merge com base no nome do município e o seu respectivo UF. Para isto, precisei padronizar os nomes, tendo sido necessário:

* Padronizar caracteres em caixa alta;
* Remover os acentos;
* Para analisar pequenas diferenças de grafia, criei função para comparar os nomes dos municípios pela distância de edição (Levenstein).

In [30]:
df_codigos_municipios = dfBolsa[['NomeMunicipio', 'UF', 'CodigoSIAFImunicipio']].copy()

In [31]:
df_codigos_municipios.head()

,NomeMunicipio,UF,CodigoSIAFImunicipio
0,CAMPINAS,SP,6291
1,BROTAS DE MACAUBAS,BA,3389
2,MANAUS,AM,0255
3,RECIFE,PE,2531
4,CARUTAPERA,MA,0755


In [32]:
len(df_codigos_municipios)

13849866

In [33]:
df_codigos_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13849866 entries, 0 to 13849865
Data columns (total 3 columns):
NomeMunicipio           object
UF                      object
CodigoSIAFImunicipio    object
dtypes: object(3)
memory usage: 317.0+ MB


In [ ]:
df_codigos_municipios.CodigoSIAFImunicipio = df_codigos_municipios.CodigoSIAFImunicipio.astype(str)

In [35]:
df_codigos_municipios.drop_duplicates(inplace=True)

In [36]:
len(df_codigos_municipios)

5570

In [45]:
df_municipios_ibge = df_municipios[['CodMunicipio','NomeMunicipio','UF']].copy()

In [46]:
df_municipios_ibge.NomeMunicipio = df_municipios_ibge.NomeMunicipio.str.upper()

In [51]:
import string

In [65]:
set_char = string.ascii_uppercase + " "

In [66]:
def find_char_acentos(x):
    output = ''
    for c in x:
        if c not in set_char:
            output += c + ' '
    if output:
        return output
    return None

In [67]:
df_municipios_ibge['acentos']=df_municipios_ibge.NomeMunicipio.apply(find_char_acentos)

In [75]:
char_acentos = set()
for item in df_municipios_ibge.acentos.unique():
    if type(item) == str:
        for c in item:
            char_acentos.add(c)


In [81]:
del df_municipios_ibge['acentos']

In [76]:
char_acentos

{' ', "'", '-', 'Á', 'Â', 'Ã', 'Ç', 'É', 'Ê', 'Í', 'Ó', 'Ô', 'Õ', 'Ú'}

In [77]:
dic_acentos={'Á':'A',
             'Â':'A',
             'Ã':'A',
             'Ç':'C',
             'É':'E',
             'Ê':'E',
             'Í':'I',
             'Ó':'O',
             'Ô':'O',
             'Õ':'O',
             'Ú':'U'}

In [78]:
def remove_acentos(x):
    output=''
    for c in x:
        if c in dic_acentos.keys():
            output += dic_acentos[c]
        else:
            output += c
    return output

In [79]:
df_municipios_ibge['NomeMunicipio']=df_municipios_ibge.NomeMunicipio.apply(remove_acentos)

In [83]:
df_municipios_ibge.head()

,CodMunicipio,NomeMunicipio,UF
564,3550308,SAO PAULO,SP
67,3304557,RIO DE JANEIRO,RJ
335,2927408,SALVADOR,BA
0,5300108,BRASILIA,DF
58,2304400,FORTALEZA,CE


In [85]:
len(df_codigos_municipios)

5570

In [152]:
df_municipios1 = pd.merge(df_municipios_ibge, df_codigos_municipios, on=['NomeMunicipio','UF'], how='outer')

In [153]:
len(df_municipios1[df_municipios1.CodMunicipio.isnull()])

67

In [154]:
len(df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()])

62

67 municípios ainda não foram relacionados após a remoção dos acentos e padronização em caixa-alta. Aí que entrou a comparação com base na distância de edição de Levenstein.

In [155]:
df_municipios1.sort_values(['NomeMunicipio','UF'], inplace=True)

In [159]:
df_municipios1['NomeAlternativo']=None

In [160]:
for i,row in df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()].iterrows():
    menor_distancia = 99
    candidato_nome = ''
    for j,row2 in df_municipios1[(df_municipios1.CodMunicipio.isnull()) & 
                                 (df_municipios1.UF==row.UF) &
                                 (df_municipios1.NomeAlternativo.isnull())].iterrows():
        distancia = Levenshtein.distance(row.NomeMunicipio, row2.NomeMunicipio)
        if distancia < menor_distancia:
            menor_distancia = distancia
            candidato_nome = row2.NomeMunicipio
            candidato_codigo = row2.CodigoSIAFImunicipio
            candidato_idx = j
    if menor_distancia < 6:
        df_municipios1['NomeAlternativo'][i] = candidato_nome
        df_municipios1['CodigoSIAFImunicipio'][i] = candidato_codigo
        df_municipios1['NomeAlternativo'][candidato_idx] = '---'
        

In [163]:
df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()]

,CodMunicipio,NomeMunicipio,UF,CodigoSIAFImunicipio,NomeAlternativo
4994,4128625,ALTO PARAISO,PR,NaN,None
3130,2401305,AUGUSTO SEVERO,RN,NaN,None
3278,4202057,BALNEARIO BARRA DO SUL,SC,NaN,None
1957,4212809,BALNEARIO PICARRAS,SC,NaN,None
1457,2607604,ILHA DE ITAMARACA,PE,NaN,None
2841,3131802,ITABIRINHA,MG,NaN,None
3168,2405306,JANUARIO CICCO,RN,NaN,None
4725,2408706,PARAU,RN,NaN,None
2822,1600154,PEDRA BRANCA DO AMAPARI,AP,NaN,None
3218,2410306,PRESIDENTE JUSCELINO,RN,NaN,None


Consegui reduzir de 67 para 14 os municípios que ainda não foram relacionados. Para isso, permiti o uso automático da distância de Levenstein quando a mesma dava um valor menor que 6.

Abaixo, criei função similar, porém supervisionada, para os casos restantes.

In [168]:
# df_municipios1[df_municipios1.CodMunicipio.isnull() & df_municipios1.NomeAlternativo.isnull()]

In [167]:
# df_municipios1[df_municipios1.NomeAlternativo.notnull()]

In [170]:
for i,row in df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()].iterrows():
    menor_distancia = 99
    candidato_nome = ''
    for j,row2 in df_municipios1[(df_municipios1.CodMunicipio.isnull()) & 
                                 (df_municipios1.UF==row.UF) &
                                 (df_municipios1.NomeAlternativo.isnull())].iterrows():
        distancia = Levenshtein.distance(row.NomeMunicipio, row2.NomeMunicipio)
        if distancia < menor_distancia:
            menor_distancia = distancia
            candidato_nome = row2.NomeMunicipio
            candidato_codigo = row2.CodigoSIAFImunicipio
            candidato_idx = j
    if menor_distancia < 50:
        if input('Aprova? (S/N) %s _x_ %s (%s)' %(row.NomeMunicipio,candidato_nome,row.UF))=='S':
            df_municipios1['NomeAlternativo'][i] = candidato_nome
            df_municipios1['CodigoSIAFImunicipio'][i] = candidato_codigo
            df_municipios1['NomeAlternativo'][candidato_idx] = '---'
            print(" Gravado com sucesso...")
        else:
            print(" Ignorando...")

Aprova? (S/N) ALTO PARAISO _x_ VILA ALTA (PR)S
 Gravado com sucesso...
Aprova? (S/N) AUGUSTO SEVERO _x_ BOA SAUDE (RN)
 Ignorando...
Aprova? (S/N) BALNEARIO BARRA DO SUL _x_ BALNEARIO RINCÃO (SC)
 Ignorando...
Aprova? (S/N) BALNEARIO PICARRAS _x_ BALNEARIO RINCÃO (SC)
 Ignorando...
Aprova? (S/N) ILHA DE ITAMARACA _x_ ITAMARACA (PE)S
 Gravado com sucesso...
Aprova? (S/N) ITABIRINHA _x_ ITABIRINHA DE MANTENA (MG)S
 Gravado com sucesso...
Aprova? (S/N) JANUARIO CICCO _x_ SERRA CAIADA (RN)
 Ignorando...
Aprova? (S/N) PARAU _x_ BOA SAUDE (RN)
 Ignorando...
Aprova? (S/N) PEDRA BRANCA DO AMAPARI _x_ AMAPARI (AP)S
 Gravado com sucesso...
Aprova? (S/N) PRESIDENTE JUSCELINO _x_ SERRA CAIADA (RN)S
 Gravado com sucesso...
Aprova? (S/N) SANTA ROSA DO PURUS _x_ SANTA ROSA (AC)S
 Gravado com sucesso...
Aprova? (S/N) SAO BENTINHO _x_ SAO BENTO DE POMBAL (PB)S
 Gravado com sucesso...
Aprova? (S/N) SAO DOMINGOS _x_ SAO DOMINGOS DE POMBAL (PB)S
 Gravado com sucesso...
Aprova? (S/N) SAO VALERIO _x_ SAO VA

Faltaram relacionar 5 municípios. Além disso, outros 5 municípios só existem em na tabela do Bolsa-Família (não existem na tabela do Censo).

In [186]:
len(df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()])

5

In [187]:
len(df_municipios1[df_municipios1.CodMunicipio.isnull() & df_municipios1.NomeAlternativo.isnull()])

5

Os municípios abaixo foram criados depois do ano de 2010, por isso não constam no Censo do IBGE:
* Pinto Bandeira (RS)
* Pescaria Brava (SC)
* Balneário Rincão (SC)
* Paraíso das Águas (MS)
* Mojuí dos Campos (PA)

In [183]:
novos_municipios = [('PINTO BANDEIRA', 'RS'),
                    ('PESCARIA BRAVA', 'SC'),
                    ('BALNEARIO RINCÃO', 'SC'),
                    ('PARAÍSO DAS ÁGUAS', 'MS'),
                    ('MOJUÍ DOS CAMPOS', 'PA')]

In [184]:
for cid, uf in novos_municipios:
    df_municipios1.loc[(df_municipios1.NomeMunicipio==cid) & (df_municipios1.UF==uf),'NomeAlternativo']='--- (novo)'

Aqui precisarei verificar através de pesquisa manual os nomes, para relacionar os municípios:

In [189]:
df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()]

,CodMunicipio,NomeMunicipio,UF,CodigoSIAFImunicipio,NomeAlternativo
3130,2401305,AUGUSTO SEVERO,RN,NaN,None
3278,4202057,BALNEARIO BARRA DO SUL,SC,NaN,None
1957,4212809,BALNEARIO PICARRAS,SC,NaN,None
3168,2405306,JANUARIO CICCO,RN,NaN,None
4725,2408706,PARAU,RN,NaN,None


In [188]:
df_municipios1[df_municipios1.CodMunicipio.isnull() & df_municipios1.NomeAlternativo.isnull()]

,CodMunicipio,NomeMunicipio,UF,CodigoSIAFImunicipio,NomeAlternativo
5605,NaN,BARRA DO SUL,SC,5549,None
5570,NaN,BOA SAUDE,RN,1703,None
5575,NaN,CAMPO GRANDE,RN,1625,None
5594,NaN,ESPIRITO SANTO DO OESTE,RN,1773,None
5613,NaN,PICARRAS,SC,8251,None


In [190]:
dic_municipios={'BALNEARIO BARRA DO SUL': 'BARRA DO SUL',
                'BALNEARIO PICARRAS': 'PICARRAS',
                'AUGUSTO SEVERO': 'CAMPO GRANDE',
                'JANUARIO CICCO': 'BOA SAUDE',
                'PARAU': 'ESPIRITO SANTO DO OESTE'}


In [202]:
for i,row in df_municipios1[df_municipios1.CodigoSIAFImunicipio.isnull()].iterrows():
    obj = df_municipios1[(df_municipios1.CodMunicipio.isnull()) & (df_municipios1.NomeMunicipio==dic_municipios[row.NomeMunicipio])]
    codigo = obj.loc[obj.index[0]].CodigoSIAFImunicipio
    nome = obj.loc[obj.index[0]].NomeMunicipio
    df_municipios1['CodigoSIAFImunicipio'][i] = codigo
    df_municipios1['NomeAlternativo'][i] = nome
    df_municipios1['NomeAlternativo'][codigo] = '---'

In [245]:
dfMunicipiosRelated = df_municipios1[(df_municipios1.CodMunicipio.notnull()) | 
                                     (df_municipios1.NomeAlternativo.str.contains('novo'))].copy()

In [246]:
dfMunicipiosRelated.shape

(5570, 5)

Aqui imprimi os casos dos municípios que usaram a menor distância de Levenstein para uma segunda conferência.

In [247]:
print(len(dfMunicipiosRelated[dfMunicipiosRelated.NomeAlternativo.notnull()]))
dfMunicipiosRelated[dfMunicipiosRelated.NomeAlternativo.notnull()]

63


,CodMunicipio,NomeMunicipio,UF,CodigoSIAFImunicipio,NomeAlternativo
558,2400208,ACU,RN,1603,ASSU
4994,4128625,ALTO PARAISO,PR,5523,VILA ALTA
4233,3102506,AMPARO DO SERRA,MG,4049,AMPARO DA SERRA
1154,3300233,ARMACAO DOS BUZIOS,RJ,0770,ARMACAO DE BUZIOS
3130,2401305,AUGUSTO SEVERO,RN,1625,CAMPO GRANDE
5612,NaN,BALNEARIO RINCÃO,SC,1192,--- (novo)
4685,4102752,BELA VISTA DA CAROBA,PR,0834,BELA VISTA DO CAROBA
1624,2601607,BELEM DO SAO FRANCISCO,PE,2331,BELEM DE SAO FRANCISCO
1553,3507803,BRODOWSKI,SP,6257,BRODOSQUI
41,3301009,CAMPOS DOS GOYTACAZES,RJ,5819,CAMPOS DOS GOITACAZES


Criei tabela relacionando cada UF a sua respectiva região do país.

In [248]:
REGIOES_BRASIL={
    'AM': 'Norte',
    'RR': 'Norte',
    'AP': 'Norte',
    'PA': 'Norte',
    'TO': 'Norte',
    'RO': 'Norte',
    'AC': 'Norte',
    'MA': 'Nordeste',
    'PI': 'Nordeste',
    'CE': 'Nordeste',
    'RN': 'Nordeste',
    'PE': 'Nordeste',
    'PB': 'Nordeste',
    'SE': 'Nordeste',
    'AL': 'Nordeste',
    'BA': 'Nordeste',
    'MT': 'Centro-Oeste',
    'MS': 'Centro-Oeste',
    'GO': 'Centro-Oeste',
    'SP': 'Sudeste',
    'RJ': 'Sudeste',
    'ES': 'Sudeste',
    'MG': 'Sudeste',
    'PR': 'Sul',
    'RS': 'Sul',
    'SC': 'Sul',
    'DF': 'Distrito Federal'}

In [249]:
dfMunicipiosRelated['Regiao']=dfMunicipiosRelated.UF.map(REGIOES_BRASIL)

In [252]:
dfMunicipiosRelated = dfMunicipiosRelated[['CodMunicipio', 'CodigoSIAFImunicipio', 'NomeMunicipio', 'UF', 'Regiao', 'NomeAlternativo']]

In [253]:
dfMunicipiosRelated.head()

,CodMunicipio,CodigoSIAFImunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo
3672,5200050,1050,ABADIA DE GOIAS,GO,Centro-Oeste,None
3715,3100104,4001,ABADIA DOS DOURADOS,MG,Sudeste,None
2081,5200100,9201,ABADIANIA,GO,Centro-Oeste,None
1437,3100203,4003,ABAETE,MG,Sudeste,None
186,1500107,0401,ABAETETUBA,PA,Norte,None


#### Inserindo coordenadas dos municípios (obtive um script SQL do blog (https://updatedcode.wordpress.com/2014/08/23/tabela-das-cidades-brasileiras/), contendo lista de municípios e suas respectivas latitude e longitude médias). Manualmente transformei este arquivo em CSV.

In [354]:
df_geo = pd.read_csv('Municipios_Brasileiros_Localizacao.csv', dtype={"Codigo": np.str})

In [355]:
df_geo.head()

,Codigo,NomeMunicipio,UF,Latitude,Longitude
0,5200050,Abadia de Goiás,GO,-16.75730,-49.4412
1,3100104,Abadia dos Dourados,MG,-18.48310,-47.3916
2,5200100,Abadiânia,GO,-16.19700,-48.7057
3,3100203,Abaeté,MG,-19.15510,-45.4444
4,1500107,Abaetetuba,PA,-1.72183,-48.8788


In [356]:
dfMunicipiosRelated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 3672 to 5063
Data columns (total 6 columns):
CodMunicipio            5565 non-null object
CodigoSIAFImunicipio    5570 non-null object
NomeMunicipio           5570 non-null object
UF                      5570 non-null object
Regiao                  5570 non-null object
NomeAlternativo         63 non-null object
dtypes: object(6)
memory usage: 304.6+ KB


In [357]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5478 entries, 0 to 5477
Data columns (total 5 columns):
Codigo           5478 non-null object
NomeMunicipio    5478 non-null object
UF               5478 non-null object
Latitude         5478 non-null float64
Longitude        5478 non-null float64
dtypes: float64(2), object(3)
memory usage: 214.1+ KB


In [376]:
dfMunicipiosRelated = dfMunicipiosRelated.merge(df_geo[['Codigo','Latitude','Longitude']], left_on="CodMunicipio", right_on="Codigo", how="left")

In [377]:
dfMunicipiosRelated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 9 columns):
CodMunicipio            5565 non-null object
CodigoSIAFImunicipio    5570 non-null object
NomeMunicipio           5570 non-null object
UF                      5570 non-null object
Regiao                  5570 non-null object
NomeAlternativo         63 non-null object
Codigo                  5478 non-null object
Latitude                5478 non-null float64
Longitude               5478 non-null float64
dtypes: float64(2), object(7)
memory usage: 435.2+ KB


In [379]:
del dfMunicipiosRelated['Codigo']

#### Obtendo o código do IBGE para cada município da tabela

In [313]:
df = dfBolsaGrouped.merge(dfMunicipiosRelated, on='CodigoSIAFImunicipio')

In [322]:
df.sort_values('ValorMedio', ascending=False).head()

,CodigoSIAFImunicipio,ValorMedio,ValorTotal,Count,CodMunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo
37,0038,371.703100,455708.0,1226,1400704,UIRAMUTA,RR,Norte,None
659,0661,366.329359,245807.0,671,1200435,SANTA ROSA DO PURUS,AC,Norte,SANTA ROSA
440,0442,359.765657,178084.0,495,2500577,ALGODAO DE JANDAIRA,PB,Nordeste,None
154,0155,329.753465,452092.0,1371,1200344,MANOEL URBANO,AC,Norte,None
258,0259,323.359613,935156.0,2892,1302801,MARAA,AM,Norte,None


#### Mesclando com os dados do Censo de cada município

Optei por ignorar os 5 novos municípios criados pós-Censo de 2010.

In [324]:
df2 = df.merge(df_municipios[['CodMunicipio','TotalPopulacao2010','TotalPopulacaoUrbana','TotalPopulacaoRural']], on='CodMunicipio')

In [328]:
df2.head()

,CodigoSIAFImunicipio,ValorMedio,ValorTotal,Count,CodMunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo,TotalPopulacao2010,TotalPopulacaoUrbana,TotalPopulacaoRural
0,0001,156.740193,583387.0,3722,1100106,GUAJARA-MIRIM,RO,Norte,None,41646,35197,6449
1,0002,144.883144,140102.0,967,1100379,ALTO ALEGRE DOS PARECIS,RO,Norte,None,12826,3665,9161
2,0003,145.421862,3517464.0,24188,1100205,PORTO VELHO,RO,Norte,None,426558,391014,35544
3,0004,139.553310,400518.0,2870,1100452,BURITIS,RO,Norte,None,32385,18113,14272
4,0005,137.725603,644969.0,4683,1100122,JI-PARANA,RO,Norte,None,116587,104841,11746


In [329]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5564
Data columns (total 12 columns):
CodigoSIAFImunicipio    5565 non-null object
ValorMedio              5565 non-null float64
ValorTotal              5565 non-null float64
Count                   5565 non-null int64
CodMunicipio            5565 non-null object
NomeMunicipio           5565 non-null object
UF                      5565 non-null object
Regiao                  5565 non-null object
NomeAlternativo         58 non-null object
TotalPopulacao2010      5565 non-null int64
TotalPopulacaoUrbana    5565 non-null int64
TotalPopulacaoRural     5565 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 565.2+ KB


Calculando o percentual da população rural.

In [331]:
df2['PercentRural'] = (df2.TotalPopulacaoRural / df2.TotalPopulacao2010) * 100

Calculando o percentual da população que recebe Bolsa-Família.

In [333]:
df2['Percent_Pop_Bolsa'] = (df2.Count / df2.TotalPopulacao2010) * 100

In [334]:
df2.head()

,CodigoSIAFImunicipio,ValorMedio,ValorTotal,Count,CodMunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo,TotalPopulacao2010,TotalPopulacaoUrbana,TotalPopulacaoRural,PercentRural,Percent_Pop_Bolsa
0,0001,156.740193,583387.0,3722,1100106,GUAJARA-MIRIM,RO,Norte,None,41646,35197,6449,15.485281,8.937233
1,0002,144.883144,140102.0,967,1100379,ALTO ALEGRE DOS PARECIS,RO,Norte,None,12826,3665,9161,71.425230,7.539373
2,0003,145.421862,3517464.0,24188,1100205,PORTO VELHO,RO,Norte,None,426558,391014,35544,8.332747,5.670507
3,0004,139.553310,400518.0,2870,1100452,BURITIS,RO,Norte,None,32385,18113,14272,44.069785,8.862128
4,0005,137.725603,644969.0,4683,1100122,JI-PARANA,RO,Norte,None,116587,104841,11746,10.074880,4.016743


Cidades com maior proporção de moradores que recebem o benefício.

In [337]:
df2.sort_values('Percent_Pop_Bolsa', ascending=False).head()

,CodigoSIAFImunicipio,ValorMedio,ValorTotal,Count,CodMunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo,TotalPopulacao2010,TotalPopulacaoUrbana,TotalPopulacaoRural,PercentRural,Percent_Pop_Bolsa
253,0254,220.995137,636245.0,2879,2111789,SERRANO DO MARANHAO,MA,Nordeste,None,10924,4222,6702,61.351153,26.354815
915,0917,209.108481,1060180.0,5070,2111003,SAO JOAO BATISTA,MA,Nordeste,None,19966,5357,14609,73.169388,25.393168
295,0296,232.339157,226066.0,973,2202455,CAPITAO GERVASIO OLIVEIRA,PI,Nordeste,None,3876,1162,2714,70.020640,25.103199
849,0851,235.995920,1099033.0,4657,2107605,PALMEIRANDIA,MA,Nordeste,None,18766,3461,15305,81.557071,24.816157
827,0829,213.761896,1136572.0,5317,2106508,MATINHA,MA,Nordeste,None,21832,8838,12994,59.518139,24.354159


In [370]:
# df2.sort_values('TotalPopulacao2010', ascending=False).head(20)

#### Analisando dados por região do país

In [392]:
PorRegiao = df2.groupby('Regiao', as_index=False).sum()

In [393]:
PorRegiao['PercentRural'] = (PorRegiao.TotalPopulacaoRural / PorRegiao.TotalPopulacao2010) * 100
PorRegiao['Percent_Pop_Bolsa'] = (PorRegiao.Count / PorRegiao.TotalPopulacao2010) * 100
PorRegiao['ValorMedio'] = PorRegiao['ValorTotal'] / PorRegiao['Count']

In [396]:
PorRegiao

,Regiao,ValorMedio,ValorTotal,Count,TotalPopulacao2010,TotalPopulacaoUrbana,TotalPopulacaoRural,PercentRural,Percent_Pop_Bolsa
0,Centro-Oeste,146.098930,9.248515e+07,633031,11487377,10003623,1483754,12.916386,5.510666
1,Distrito Federal,139.652246,1.124312e+07,80508,2562963,2476249,86714,3.383350,3.141208
2,Nordeste,166.965746,1.167032e+09,6989648,53078137,38816895,14261242,26.868392,13.168601
3,Norte,183.599025,3.142850e+08,1711801,15865678,11663184,4202494,26.487957,10.789334
4,Sudeste,148.106852,5.210159e+08,3517838,80353724,74661877,5691847,7.083489,4.377940
5,Sul,143.177059,1.307756e+08,913384,27384815,23257880,4126935,15.070158,3.335367


In [555]:
PorRegiao[['Regiao','ValorMedio','Count','TotalPopulacao2010','PercentRural','Percent_Pop_Bolsa']]

,Regiao,ValorMedio,Count,TotalPopulacao2010,PercentRural,Percent_Pop_Bolsa
0,Centro-Oeste,146.098930,633031,11487377,12.916386,5.510666
1,Distrito Federal,139.652246,80508,2562963,3.383350,3.141208
2,Nordeste,166.965746,6989648,53078137,26.868392,13.168601
3,Norte,183.599025,1711801,15865678,26.487957,10.789334
4,Sudeste,148.106852,3517838,80353724,7.083489,4.377940
5,Sul,143.177059,913384,27384815,15.070158,3.335367


#### Analisando dados por estado

In [398]:
PorEstado = df2.groupby('UF', as_index=False).sum()

In [399]:
PorEstado['PercentRural'] = (PorEstado.TotalPopulacaoRural / PorEstado.TotalPopulacao2010) * 100
PorEstado['Percent_Pop_Bolsa'] = (PorEstado.Count / PorEstado.TotalPopulacao2010) * 100
PorEstado['ValorMedio'] = PorEstado['ValorTotal'] / PorEstado['Count']

In [401]:
PorEstado.head()

,UF,ValorMedio,ValorTotal,Count,TotalPopulacao2010,TotalPopulacaoUrbana,TotalPopulacaoRural,PercentRural,Percent_Pop_Bolsa
0,AC,232.619621,19568893.0,84124,732793,532080,200713,27.390136,11.479913
1,AL,167.221845,66820846.0,399594,3120922,2298091,822831,26.364997,12.803716
2,AM,200.070580,74055325.0,370146,3480937,2755756,725181,20.832925,10.633516
3,AP,188.221802,11836328.0,62885,668689,600561,68128,10.188294,9.404222
4,BA,162.385297,297585672.0,1832590,14021432,10105218,3916214,27.930200,13.069920


#### Ranking dos estados pelo percentual da população que recebe o benefício.

In [556]:
PorEstado.sort_values('Percent_Pop_Bolsa', ascending=False)[['UF','ValorMedio','Count','TotalPopulacao2010','PercentRural','Percent_Pop_Bolsa']]

,UF,ValorMedio,Count,TotalPopulacao2010,PercentRural,Percent_Pop_Bolsa
9,MA,187.753457,970358,6569683,36.926515,14.770241
16,PI,183.720743,454115,3119015,34.231929,14.559564
14,PB,175.053985,523869,3766834,24.631614,13.907409
24,SE,151.224743,272351,2068031,26.488384,13.169580
4,BA,162.385297,1832590,14021432,27.930200,13.069920
15,PE,158.932724,1130656,8796032,19.851724,12.854160
1,AL,167.221845,399594,3120922,26.364997,12.803716
5,CE,159.787325,1052094,8448055,24.905910,12.453683
13,PA,179.787629,909523,7588078,31.509428,11.986210
0,AC,232.619621,84124,732793,27.390136,11.479913


In [403]:
PorEstado.sort_values('ValorMedio', ascending=False)

,UF,ValorMedio,ValorTotal,Count,TotalPopulacao2010,TotalPopulacaoUrbana,TotalPopulacaoRural,PercentRural,Percent_Pop_Bolsa
0,AC,232.619621,19568893.0,84124,732793,532080,200713,27.390136,11.479913
2,AM,200.070580,74055325.0,370146,3480937,2755756,725181,20.832925,10.633516
3,AP,188.221802,11836328.0,62885,668689,600561,68128,10.188294,9.404222
9,MA,187.753457,182188069.0,970358,6569683,4143728,2425955,36.926515,14.770241
16,PI,183.720743,83430345.0,454115,3119015,2051316,1067699,34.231929,14.559564
21,RR,180.033571,9047047.0,50252,451227,344780,106447,23.590565,11.136745
13,PA,179.787629,163520984.0,909523,7588078,5197118,2390960,31.509428,11.986210
14,PB,175.053985,91705356.0,523869,3766834,2839002,927832,24.631614,13.907409
1,AL,167.221845,66820846.0,399594,3120922,2298091,822831,26.364997,12.803716
26,TO,164.063461,22207466.0,135359,1383453,1090241,293212,21.194215,9.784142


------


#### Lendo o arquivo do Bolsa-Família de 5 anos atrás (06/2011)

A minha ideia era também explorar informações a respeito das mudanças que ocorreram entre as amostras nesse período de 5 anos.

In [450]:
dfBolsa2011 = pd.read_csv('201106_BolsaFamiliaFolhaPagamento.csv',
                      sep='\t',
                      header=(0),
                      encoding='cp1252',
                      dtype={"Valor Parcela": np.str,
                             "Código SIAFI Município": np.str})

dfBolsa2011.rename(columns={"Código SIAFI Município": "CodigoSIAFImunicipio",
                            "Nome Município": "NomeMunicipio",
                            "Código Função": "CodigoFuncao",
                            "Código Subfunção": "CodigoSubfuncao",
                            "Código Programa": "CodigoPrograma",
                            "Código Ação": "CodigoAcao",
                            "NIS Favorecido": "NISFavorecido",
                            "Nome Favorecido": "NomeFavorecido",
                            "Fonte-Finalidade": "FonteFinalidade",
                            "Valor Parcela": "ValorParcela",
                            "Mês Competência": "MesCompetencia"}, inplace=True)

dfBolsa2011.ValorParcela = dfBolsa2011.ValorParcela.str.replace(",","")
dfBolsa2011.ValorParcela = dfBolsa2011.ValorParcela.apply(pd.to_numeric)
del dfBolsa2011['FonteFinalidade']
dfBolsa2011.head()

dfBolsaTmp2 = dfBolsa2011.groupby('CodigoSIAFImunicipio')
dfBolsa2011Grouped = dfBolsaTmp2['ValorParcela'].agg({'ValorTotal':np.sum, 'ValorMedio':np.mean})
dfBolsa2011Grouped['Count'] = dfBolsaTmp2.size()
dfBolsa2011Grouped.reset_index(inplace=True)
dfBolsa2011Grouped.rename(columns={'ValorMedio': 'ValorMedio2011',
                                   'ValorTotal': 'ValorTotal2011',
                                   'Count': 'Count2011'}, inplace=True)

df2 = df2.merge(dfBolsa2011Grouped, on='CodigoSIAFImunicipio', how='left')
df2.rename(columns={'ValorMedio': 'ValorMedio2016',
                    'ValorTotal': 'ValorTotal2016',
                    'Count': 'QtdeRepasses2016',
                    'PercentRural': 'PercentPopulacaoRural2010',
                    'Percent_Pop_Bolsa': 'PercentPopulacaoBolsa2016',
                    'Count2011': 'QtdeRepasses2011'}, inplace=True)


In [455]:
PorMunicipio = df2[['CodigoSIAFImunicipio', 'CodMunicipio', 'NomeMunicipio', 'UF', 'Regiao', 'NomeAlternativo',
                    'TotalPopulacao2010', 'PercentPopulacaoRural2010',
                    'ValorMedio2016', 'ValorTotal2016', 'QtdeRepasses2016', 'PercentPopulacaoBolsa2016',
                    'ValorMedio2011', 'ValorTotal2011', 'QtdeRepasses2011']].copy()

In [456]:
PorMunicipio['PercentPopulacaoBolsa2011'] = (PorMunicipio['QtdeRepasses2011']/PorMunicipio['TotalPopulacao2010'])*100

In [458]:
PorMunicipio.head()

,CodigoSIAFImunicipio,CodMunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo,TotalPopulacao2010,PercentPopulacaoRural2010,ValorMedio2016,ValorTotal2016,QtdeRepasses2016,PercentPopulacaoBolsa2016,ValorMedio2011,ValorTotal2011,QtdeRepasses2011,PercentPopulacaoBolsa2011
0,0001,1100106,GUAJARA-MIRIM,RO,Norte,None,41646,15.485281,156.740193,583387.0,3722,8.937233,126.622605,455968.0,3601,8.646689
1,0002,1100379,ALTO ALEGRE DOS PARECIS,RO,Norte,None,12826,71.425230,144.883144,140102.0,967,7.539373,117.087745,161464.0,1379,10.751598
2,0003,1100205,PORTO VELHO,RO,Norte,None,426558,8.332747,145.421862,3517464.0,24188,5.670507,122.481202,2681236.0,21891,5.132010
3,0004,1100452,BURITIS,RO,Norte,None,32385,44.069785,139.553310,400518.0,2870,8.862128,117.510664,385670.0,3282,10.134321
4,0005,1100122,JI-PARANA,RO,Norte,None,116587,10.074880,137.725603,644969.0,4683,4.016743,104.328395,669058.0,6413,5.500613


In [459]:
PorMunicipio['Diferenca2016para2011']=PorMunicipio['PercentPopulacaoBolsa2016']-PorMunicipio['PercentPopulacaoBolsa2011']

In [462]:
PorMunicipio.sort_values('Diferenca2016para2011', ascending=False).head()

,CodigoSIAFImunicipio,CodMunicipio,NomeMunicipio,UF,Regiao,NomeAlternativo,TotalPopulacao2010,PercentPopulacaoRural2010,ValorMedio2016,ValorTotal2016,QtdeRepasses2016,PercentPopulacaoBolsa2016,ValorMedio2011,ValorTotal2011,QtdeRepasses2011,PercentPopulacaoBolsa2011,Diferenca2016para2011
1176,1180,2206720,NAZARIA,PI,Nordeste,None,8039,79.450180,195.472783,319598.0,1635,20.338351,103.448819,39414.0,381,4.739395,15.598955
525,0527,1506401,SANTA CRUZ DO ARARI,PA,Norte,None,8163,50.986157,203.234396,306071.0,1506,18.449100,120.627660,68034.0,564,6.909225,11.539875
653,0655,1200351,MARECHAL THAUMATURGO,AC,Norte,None,14200,72.035211,288.373374,753808.0,2614,18.408451,146.794619,163676.0,1115,7.852113,10.556338
667,0669,1600253,ITAUBAL,AP,Norte,None,4267,58.612608,257.200696,221707.0,862,20.201547,131.985577,54906.0,416,9.749238,10.452308
535,0537,1506906,SANTAREM NOVO,PA,Norte,None,6145,70.496338,178.422642,236410.0,1325,21.562246,129.760218,95244.0,734,11.944670,9.617575


In [478]:
PorMunicipio = PorMunicipio.merge(dfMunicipiosRelated[['CodMunicipio','Latitude','Longitude']], on='CodMunicipio', how='left')

In [479]:
PorMunicipio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5565 entries, 0 to 5564
Data columns (total 19 columns):
CodigoSIAFImunicipio         5565 non-null object
CodMunicipio                 5565 non-null object
NomeMunicipio                5565 non-null object
UF                           5565 non-null object
Regiao                       5565 non-null object
NomeAlternativo              58 non-null object
TotalPopulacao2010           5565 non-null int64
PercentPopulacaoRural2010    5565 non-null float64
ValorMedio2016               5565 non-null float64
ValorTotal2016               5565 non-null float64
QtdeRepasses2016             5565 non-null int64
PercentPopulacaoBolsa2016    5565 non-null float64
ValorMedio2011               5565 non-null float64
ValorTotal2011               5565 non-null float64
QtdeRepasses2011             5565 non-null int64
PercentPopulacaoBolsa2011    5565 non-null float64
Diferenca2016para2011        5565 non-null float64
Latitude                     5478 non-nu

In [559]:
PorMunicipio.sort_values('PercentPopulacaoBolsa2016', ascending=False)[['NomeMunicipio','UF','TotalPopulacao2010','PercentPopulacaoBolsa2016']]

,NomeMunicipio,UF,TotalPopulacao2010,PercentPopulacaoBolsa2016
253,SERRANO DO MARANHAO,MA,10924,26.354815
915,SAO JOAO BATISTA,MA,19966,25.393168
295,CAPITAO GERVASIO OLIVEIRA,PI,3876,25.103199
849,PALMEIRANDIA,MA,18766,24.816157
827,MATINHA,MA,21832,24.354159
2164,GRACHO CARDOSO,SE,5648,24.061615
364,PAVUSSU,PI,3666,24.058920
368,NOVA SANTA RITA,PI,4192,24.021947
386,SAO JOAO DO ARRAIAL,PI,7337,24.015265
275,BELEM DO PIAUI,PI,3284,23.995128


In [602]:
# PorMunicipio[PorMunicipio.PercentPopulacaoBolsa2016 > 20][['NomeMunicipio','UF','TotalPopulacao2010','PercentPopulacaoBolsa2016']]

Gerando dados em JSON para geração do mapa:

In [566]:
PorEstado[['UF','Percent_Pop_Bolsa']].set_index('UF').to_json('html/uf_bolsafamilia.json')

### Mapa - Percentual da população que recebe o Bolsa-Família por estado

Para gerar o mapa abaixo, utilizei as bibliotecas do D3.js. Obtive os Shapefiles do Brasil do GADM (http://www.gadm.org/country), e os converti para TopoJSON no MapShaper (http://www.mapshaper.org/), também os simplificando (arquivo JSON ficou com 69 KB, sendo que o arquivo original SHP tinha 9 MB).

Me baseei no exemplo de um blog (http://blog.superquadra.co/frontend/mapa-do-brasil-em-d3js/). Tive que escrever JavaScript para fazer a leitura do JSON e a criação de uma função para personalizar a escala de cores de acordo com os dados (percentual da população que recebe o Bolsa-Família). 

In [600]:
%%HTML
<iframe src="./html/mapa_estados_brasil.html" width="100%" height="650px"></iframe>

Tentei gerar mapa similar, mas por município, mas não houve tempo hábil para realizar os ajustes que seriam necessários.

In [604]:
PorMunicipio[PorMunicipio.Latitude.notnull()][['NomeMunicipio','UF','Latitude','Longitude']].to_json('html/municipios_geo.json')

O arquivo html exibido abaixo está apenas lendo a lista de municípios e suas respectivas coordenadas geográficas, e as plotando no mapa.

Gostaria de ter ajustado o tamanho dos circulos de acordo com o tamanho da população de cada cidade, e a cor de acordo com o percentual de moradores que recebem o Bolsa-Família, mas não tive tempo para concluir isto - a meu ver, inicialmente seria necessário tornar o mapa escalável para melhorar a qualidade da visualização.

In [605]:
%%HTML
<iframe src="./html/municipios_geo.html" width="100%" height="650px"></iframe>